In [ ]:
import pandas as pd   

# Reemplaza 'archivo.csv' con el nombre o la ruta de tu archivo CSV
ruta_archivo = r"C:\mapariz\PROYECTOS\DataScience\Proyecto Pymes\raw_data\Personas Jurídicas y Empresas\PUB_EMPRESAS.CSV"
df = pd.read_csv(ruta_archivo", encoding='utf-8')  # Usa encoding='latin1' si tienes caracteres especiales

# Verifica las primeras filas
print(df.head())

In [6]:
print(df.columns)

Index(['Año comercial', 'RUT', 'DV', 'Razón social', 'Tramo según ventas',
       'Número de trabajadores dependie', 'Fecha inicio de actividades vige',
       'Fecha término de giro', 'Fecha primera inscripción de ac',
       'Tipo término de giro', 'Tipo de contribuyente',
       'Subtipo de contribuyente', 'Tramo capital propio positivo',
       'Tramo capital propio negativo', 'Rubro económico',
       'Subrubro económico', 'Actividad económica', 'Región', 'Provincia',
       'Comuna', 'R_PRESUNTA', 'OTROS_REGIMENES', 'Anno_comercial'],
      dtype='object')


In [9]:
# Asegura que estas columnas sean numéricas o convertibles si aplica
df['Anno_comercial'] = pd.to_numeric(df['Año comercial'], errors='coerce')
df['Numero_trabajadores_dependientes'] = pd.to_numeric(df['Número de trabajadores dependie'], errors='coerce')
df['Tramo_segun_ventas'] = pd.to_numeric(df['Tramo según ventas'], errors='coerce')  # Solo si son valores numéricos

# Columnas informativas (las que se mantienen como 'first')
columnas_informativas = [
    'DV', 'Razón social',
       'Fecha inicio de actividades vige',
       'Fecha término de giro', 'Fecha primera inscripción de ac',
       'Tipo término de giro', 'Tipo de contribuyente',
       'Subtipo de contribuyente', 'Tramo capital propio positivo',
       'Tramo capital propio negativo', 'Rubro económico',
       'Subrubro económico', 'Actividad económica', 'Región', 'Provincia',
       'Comuna', 'R_PRESUNTA', 'OTROS_REGIMENES'
]

# Diccionario de agregación
agg_dict = {col: 'first' for col in columnas_informativas}
agg_dict.update({
    'Anno_comercial': ['min', 'max'],
    'Numero_trabajadores_dependientes': ['min', 'max'],
    'Tramo_segun_ventas': ['min', 'max']
})

# Aplicar agrupamiento
agrupado = df.groupby('RUT').agg(agg_dict)

# Aplanar columnas
agrupado.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in agrupado.columns]

# Renombrar columnas para mayor claridad
agrupado = agrupado.rename(columns={
    'Anno_comercial_min': 'Anno_min',
    'Anno_comercial_max': 'Anno_max',
    'Numero_trabajadores_dependientes_min': 'Trabajadores_min',
    'Numero_trabajadores_dependientes_max': 'Trabajadores_max',
    'Tramo_segun_ventas_min': 'Ventas_min',
    'Tramo_segun_ventas_max': 'Ventas_max'
})

# Calcular número de años
agrupado['Numero_de_annos'] = agrupado['Anno_max'] - agrupado['Anno_min'] + 1

# Reset index
agrupado = agrupado.reset_index()

# Mostrar resultado
print(agrupado)


              RUT DV_first                      Razón social_first  \
0        47003012        7             TACONIC  OPPORTUITY FUND LP   
1        47003030        5                 CAPLIMIT INVESTMENTS BV   
2        47003194        8          RUT CUSTODIO LARRAIN VIAL S A.   
3        47003969        8          RUT CUSTODIO LARRAIN VIAL S A.   
4        47012566        7        BBVA CORREDORA DE BOLSA LIMITADA   
...           ...      ...                                     ...   
1558275  99599800        9                  AUTOMOTRIZ P.G CAR S.A   
1558276  99599810        6  COMPANIA DE TRANSPORTES INTEGRADOS S A   
1558277  99599830        0   INMOBILIARIA E INVERSIONES PUCARA S A   
1558278  99599850        5                SOLUCIONES ABIERTAS  S A   
1558279  99599860        2                                ORAND SA   

        Fecha inicio de actividades vige_first Fecha término de giro_first  \
0                                         None                        None   
1  

In [13]:
print(agrupado.columns)
print(agrupado)

Index(['RUT', 'DV_x_first', 'COD_SUBTIPO_first', 'RAZON_SOCIAL_first',
       'FECHA_INICIO_VIG_first', 'FECHA_TG_VIG_first', 'DV_y_first',
       'Razon_social_first', 'Fecha_inicio_actividades_vigente_first',
       'Fecha_termino_giro_first', 'Fecha_primera_inscripcion_ac_first',
       'Tipo_termino_giro_first', 'Tipo_contribuyente_first',
       'Subtipo_contribuyente_first', 'Anno_min', 'Anno_max',
       'Trabajadores_min', 'Trabajadores_max', 'Ventas_min', 'Ventas_max',
       'Numero_de_annos'],
      dtype='object')
             RUT DV_x_first  COD_SUBTIPO_first  \
0       47003012          7                414   
1       47003969          8                414   
2       50000100          3                311   
3       50000430          4                311   
4       50000510          6                311   
...          ...        ...                ...   
360227  99599800          9                213   
360228  99599810          6                213   
360229  99599830  

In [10]:
# Por ejemplo, si quieres filtrar el RUT '12345678-9'
rut_especifico = 50032710

df_rut = agrupado[agrupado['RUT'] == rut_especifico]

# Ver las primeras filas del resultado
print(df_rut.head())


          RUT DV_first                        Razón social_first  \
106  50032710        3  CENTRO DE PERFECCIONAMIENTO TECNICO LTDA   

    Fecha inicio de actividades vige_first Fecha término de giro_first  \
106                             1993-01-01                        None   

    Fecha primera inscripción de ac_first Tipo término de giro_first  \
106                            1993-01-01                       None   

    Tipo de contribuyente_first Subtipo de contribuyente_first  \
106  PERSONA JURIDICA COMERCIAL  SOC. RESPONSABILIDAD LIMITADA   

     Tramo capital propio positivo_first  ...  Comuna_first R_PRESUNTA_first  \
106                                  9.0  ...   PROVIDENCIA             None   

                                 OTROS_REGIMENES_first Anno_min Anno_max  \
106  CONTRIBUYENTE ACOGIDO AL REGIMEN DE RENTA ATRI...     2005     2023   

    Trabajadores_min Trabajadores_max Ventas_min Ventas_max  Numero_de_annos  
106               21               33      

In [12]:
print(agrupado)

              RUT DV_first                      Razón social_first  \
0        47003012        7             TACONIC  OPPORTUITY FUND LP   
1        47003030        5                 CAPLIMIT INVESTMENTS BV   
2        47003194        8          RUT CUSTODIO LARRAIN VIAL S A.   
3        47003969        8          RUT CUSTODIO LARRAIN VIAL S A.   
4        47012566        7        BBVA CORREDORA DE BOLSA LIMITADA   
...           ...      ...                                     ...   
1558275  99599800        9                  AUTOMOTRIZ P.G CAR S.A   
1558276  99599810        6  COMPANIA DE TRANSPORTES INTEGRADOS S A   
1558277  99599830        0   INMOBILIARIA E INVERSIONES PUCARA S A   
1558278  99599850        5                SOLUCIONES ABIERTAS  S A   
1558279  99599860        2                                ORAND SA   

        Fecha inicio de actividades vige_first Fecha término de giro_first  \
0                                         None                        None   
1  

In [13]:
agrupado.to_csv(r"C:\mapariz\PROYECTOS\DataScience\Proyecto Pymes\raw_data\Personas Jurídicas y Empresas\PUB_EMPRESAS_AGRUPADO.CSV", index=False, encoding='utf-8')